In [1]:
import requests as req
import pandas as pd

```
import asyncio

from chessdotcom.aio import get_player_profile, Client
#or
from chessdotcom import Client
Client.aio = True

usernames = ["fabianocaruana", "GMHikaruOnTwitch", "MagnusCarlsen", "GarryKasparov"]

cors = [get_player_profile(name) for name in usernames]

async def gather_cors(cors):
   responses = await asyncio.gather(*cors)
   return responses

responses = asyncio.run(gather_cors(cors))
```

In [2]:
headers = {
    "User-Agent":"duduardo45"
}

users = {
    'MrBarros': {
        'url':'https://api.chess.com/pub/player/mrbarros'
    },
}

ratings = {}

# api_url = "https://api.chess.com/pub/player/"

In [3]:
def get(s):
    return req.get(s, headers = headers)

# acessa os {jogador}/games/archives, pega todos os links

In [4]:
archives = "/games/archives"

u = list(users.keys())[0]

response = get(f'{users[u]['url']}{archives}')
datas = response.json()
datas = datas['archives']

# acessa cada um dos links e baixa o pgn de resposta de todos, acumulando ele em um único pgn por usuario ao terminar muda o valor no dicionario de users para True

In [5]:
player_games = []

for data in datas:
    response = get(data)
    monthly_games = response.json()['games']
    player_games.extend(monthly_games)

player_games = pd.DataFrame(player_games)
player_games['player'] = u

In [6]:
print(player_games.iloc[-1]['white'].keys())

dict_keys(['rating', 'result', '@id', 'username', 'uuid'])


In [7]:
def extract_black_white(row):

    # separating the variables to be used
    white = row['white']
    black = row['black']
    wplayer = white['username']
    bplayer = black['username']
    wresult = white['result']
    bresult = black['result']
    wrating = white['rating']
    brating = black['rating']
    player_pieces = pd.NA

    # extracting the information relative to the main player
    if wplayer == row['player']:
        if bplayer not in users.keys():
            users[bplayer] = {'url' : black['@id']}
        prating = wrating
        presult = wresult
        player_pieces = 'white'
        opponent = bplayer
        orating = brating
    elif bplayer == row['player']:
        if wplayer not in users.keys():
            users[wplayer] = {'url' : white['@id']}
        prating = brating
        presult = bresult
        player_pieces = 'black'
        opponent = wplayer
        orating = wrating
    if wresult == 'win':
        winner = 'white'
        method = bresult
    elif bresult == 'win':
        winner = 'black'
        method = wresult
    elif wresult == bresult:
        winner = 'draw'
        method = presult
    else:
        winner = 'unknown'
        method = wresult + ' + ' + bresult


    # adding the information to the DataFrame
    row['player_rating'] = prating
    row['opponent'] = opponent
    row['opponent_rating'] = orating
    row['player_pieces'] = player_pieces
    row['winner'] = winner
    row['win_method'] = method

    return row


users[u]['gathered'] = True

columns = [
    'player',
    'player_rating',
    'opponent',
    'opponent_rating',
    'player_pieces',
    'winner',
    'win_method',
]

In [8]:
player_games = player_games.apply(extract_black_white,axis=1)

In [9]:
player_games.drop(columns=['white','black'],inplace=True)

Falta:
- adicionar o player_games a um .csv externo; 
- adicionar informações externas do jogador como:
  - rating mais recente para cada modalidade;
  - ultima atualização do perfil/jogo jogado mais recente;

In [12]:
import json

print(json.dumps(users,indent=4))
print(len(users))

{
    "MrBarros": {
        "url": "https://api.chess.com/pub/player/mrbarros",
        "gathered": true
    },
    "Jalal1221": {
        "url": "https://api.chess.com/pub/player/jalal1221"
    },
    "MariaHas": {
        "url": "https://api.chess.com/pub/player/mariahas"
    },
    "Harinanda": {
        "url": "https://api.chess.com/pub/player/harinanda"
    },
    "Butterontoast": {
        "url": "https://api.chess.com/pub/player/butterontoast"
    },
    "Dangerous_adversary": {
        "url": "https://api.chess.com/pub/player/dangerous_adversary"
    },
    "crdow": {
        "url": "https://api.chess.com/pub/player/crdow"
    },
    "StefanKalafatic": {
        "url": "https://api.chess.com/pub/player/stefankalafatic"
    },
    "balta777": {
        "url": "https://api.chess.com/pub/player/balta777"
    },
    "alexandreveilleux": {
        "url": "https://api.chess.com/pub/player/alexandreveilleux"
    },
    "clementherel": {
        "url": "https://api.chess.com/pub/player

# recupera todos os usernames unicos contra o quem o jogador jogou, salva eles no dicionario com valor False, para registrar que não foi pego os arquivos deste user

# procura algum usuario no dicionario para o qual não foi feito o processo e repete para este

# adiciona o usuario ao json users.json e salva a alteração

### falta decidir o que fazer com o pgn de cada usuário; considerar adicionar o uuid de cada jogo para não salvar o dobro de dados